##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFLite 코드를 TF2로 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/tflite"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

[TensorFlow Lite](https://www.tensorflow.org/lite/guide)(TFLite)는 개발자가 장치(모바일, 임베디드, IoT 장치)에서 ML 추론을 실행하도록 돕는 도구 세트입니다. [TFLite 변환기](https://www.tensorflow.org/lite/convert)는 기존 TF 모델을 장치에서 효율적으로 실행할 수 있는 최적화된 TFLite 모델 형식으로 변환하는 도구입니다.

이 문서에서는 TF를 TFLite로 변환하기 위해 변경해야 하는 변환 코드에 대해 배우고 동일한 작업을 수행하는 몇 가지 예제를 살펴봅니다.

## TF를 TFLite로 변환하기 위해 변경해야 하는 변환 코드

- 레거시 TF1 모델 형식(Keras 파일, 고정된 GraphDef, 체크포인트, tf.Session 등)을 사용하는 경우 TF1/TF2 SavedModel로 업데이트하고 TF2 변환기 API `tf.lite.TFLiteConverter.from_saved_model(...)`를 사용하여 TFLite 모델로 변환합니다(표 1 참조).

- 변환기 API 플래그를 업데이트합니다(표 2 참조).

- `tflite.constants`와 같은 레거시 API를 제거합니다(예: `tf.lite.constants.INT8`을 `tf.int8`로 교체).

// 표 1 // TFLite Python 변환기 API 업데이트

TF1 API | TF2 API
--- | ---
`tf.lite.TFLiteConverter.from_saved_model('saved_model/',..)` | *지원됨*
`tf.lite.TFLiteConverter.from_keras_model_file('model.h5',..)` | *제거됨(SavedModel 형식으로 업데이트)*
`tf.lite.TFLiteConverter.from_frozen_graph('model.pb',..)` | *제거됨(SavedModel 형식으로 업데이트)*
`tf.lite.TFLiteConverter.from_session(sess,...)` | *제거됨(SavedModel 형식으로 업데이트)*

&lt;style&gt;   .table {margin-left: 0 !important;} &lt;/style&gt;

// 표 2 // TFLite Python 변환기 API 플래그 업데이트

TF1 API | TF2 API
--- | ---
`allow_custom_ops`<br>`optimizations`<br> `representative_dataset`<br>`target_spec` <br>`inference_input_type`<br>`inference_output_type`<br>`experimental_new_converter`<br> `experimental_new_quantizer` | *지원됨* <br><br><br><br><br><br><br><br>
`input_tensors`<br>`output_tensors`<br>`input_arrays_with_shape`<br>`output_arrays`<br>`experimental_debug_info_func` | *제거됨(지원되지 않는 변환기 API 인수)*<br><br><br><br><br>
`change_concat_input_ranges`<br>`default_ranges_stats`<br>`get_input_arrays()`<br>`inference_type`<br>`quantized_input_stats`<br> `reorder_across_fake_quant` | *제거됨(지원되지 않는 양자화 워크플로)*<br><br><br><br><br><br>
`conversion_summary_dir`<br>`dump_graphviz_dir`<br>`dump_graphviz_video` | *제거됨(대신 [네트론](https://lutzroeder.github.io/netron/) 또는 [visualize.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/tools/visualize.py)를 사용하여 모델을 시각화)*<br><br><br>
`output_format`<br>`drop_control_dependency` | *제거됨(TF2에서 지원되지 않는 특성)*<br><br>

## 예제

이제 레거시 TF1 모델을 TF1/TF2 SavedModels로 변환한 다음 TF2 TFLite 모델로 변환하는 몇 가지 예제를 살펴보겠습니다.

### 설치하기

필요한 TensorFlow 가져오기로 시작합니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

import shutil
def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass

필요한 모든 TF1 모델 형식을 생성합니다.

In [ ]:
# Create a TF1 SavedModel
SAVED_MODEL_DIR = "tf_saved_model/"
remove_dir(SAVED_MODEL_DIR)
with tf1.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=(3,), name='input')
    output = input + 2
    # print("result: ", sess.run(output, {input: [0., 2., 4.]}))
    tf1.saved_model.simple_save(
        sess, SAVED_MODEL_DIR,
        inputs={'input': input}, 
        outputs={'output': output})
print("TF1 SavedModel path: ", SAVED_MODEL_DIR)

# Create a TF1 Keras model
KERAS_MODEL_PATH = 'tf_keras_model.h5'
model = tf1.keras.models.Sequential([
    tf1.keras.layers.InputLayer(input_shape=(128, 128, 3,), name='input'),
    tf1.keras.layers.Dense(units=16, input_shape=(128, 128, 3,), activation='relu'),
    tf1.keras.layers.Dense(units=1, name='output')
])
model.save(KERAS_MODEL_PATH, save_format='h5')
print("TF1 Keras Model path: ", KERAS_MODEL_PATH)

# Create a TF1 frozen GraphDef model
GRAPH_DEF_MODEL_PATH = tf.keras.utils.get_file(
    'mobilenet_v1_0.25_128',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.25_128_frozen.tgz',
    untar=True,
) + '/frozen_graph.pb'

print("TF1 frozen GraphDef path: ", GRAPH_DEF_MODEL_PATH)

### 1. TF1 SavedModel을 TFLite 모델로 변환하기


#### 전: TF1로 변환하기

다음은 TF1 스타일의 TFlite 변환에 사용하는 일반적인 코드입니다.


In [ ]:
converter = tf1.lite.TFLiteConverter.from_saved_model(
    saved_model_dir=SAVED_MODEL_DIR,
    input_arrays=['input'],
    input_shapes={'input' : [3]}
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()
# Ignore warning: "Use '@tf.function' or '@defun' to decorate the function."

#### 후: TF2로 변환하기

더 작은 v2 변환기 플래그가 설정을 사용하여 TF1 SavedModel을 TFLite 모델로 직접 변환합니다.

In [ ]:
# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir=SAVED_MODEL_DIR)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

### 2. TF1 Keras 모델 파일을 TFLite 모델로 변환하기

#### 전: TF1로 변환하기

다음은 TF1 스타일의 TFlite 변환에 사용하는 일반적인 코드입니다.

In [ ]:
converter = tf1.lite.TFLiteConverter.from_keras_model_file(model_file=KERAS_MODEL_PATH)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### 후: TF2로 변환하기

먼저 TF1 Keras 모델 파일을 TF2 SavedModel로 변환한 다음 더 작은 v2 변환기 플래그 설정을 사용하여 TFLite 모델로 변환합니다.

In [ ]:
# Convert TF1 Keras model file to TF2 SavedModel.
model = tf.keras.models.load_model(KERAS_MODEL_PATH)
model.save(filepath='saved_model_2/')

# Convert TF2 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_2/')
tflite_model = converter.convert()

### 3. TF1 고정 GraphDef를 TFLite 모델로 변환하기


#### 전: TF1로 변환하기

다음은 TF1 스타일의 TFlite 변환에 사용하는 일반적인 코드입니다.

In [ ]:
converter = tf1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=GRAPH_DEF_MODEL_PATH,
    input_arrays=['input'],
    input_shapes={'input' : [1, 128, 128, 3]},
    output_arrays=['MobilenetV1/Predictions/Softmax'],
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### 후: TF2로 변환하기

먼저 TF1 고정 GraphDef를 TF1 SavedModel로 변환한 다음 더 작은 v2 변환기 플래그 설정을 사용하여 TFLite 모델로 변환합니다.


In [ ]:
## Convert TF1 frozen Graph to TF1 SavedModel.

# Load the graph as a v1.GraphDef
import pathlib
gdef = tf.compat.v1.GraphDef()
gdef.ParseFromString(pathlib.Path(GRAPH_DEF_MODEL_PATH).read_bytes())

# Convert the GraphDef to a tf.Graph
with tf.Graph().as_default() as g:
  tf.graph_util.import_graph_def(gdef, name="")

# Look up the input and output tensors.
input_tensor = g.get_tensor_by_name('input:0') 
output_tensor = g.get_tensor_by_name('MobilenetV1/Predictions/Softmax:0')

# Save the graph as a TF1 Savedmodel
remove_dir('saved_model_3/')
with tf.compat.v1.Session(graph=g) as s:
  tf.compat.v1.saved_model.simple_save(
      session=s,
      export_dir='saved_model_3/',
      inputs={'input':input_tensor},
      outputs={'output':output_tensor})

# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_3/')
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

# 추가 자료

- 워크플로 및 최신 특성에 대한 자세한 내용은 [TFLite 가이드](https://www.tensorflow.org/lite/guide)를 참고하세요.
- TF1 코드 또는 레거시 TF1 모델 형식(Keras `.h5` 파일, 고정 GraphDef `.pb` 등)을 사용하는 경우 코드를 업데이트하고 모델을 [TF2 SavedModel 모델 형식](https://www.tensorflow.org/guide/saved_model)으로 마이그레이션하세요.
